In [1]:
from datasets import load_dataset
from CSEcollator import CSECollator
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import torch.nn as nn
from tqdm import tqdm
#from transformers import *
from transformers.models.bert.modeling_bert import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding,TrainingArguments, Trainer
batch_size = 64

In [2]:
def load_data(data_files, tokenizer, batch_size, max_length):
    ds = load_dataset("text", data_files=data_files)
    ds_tokenized = ds.map(lambda example: tokenizer(example["text"], truncation=True, max_length=max_length, padding='max_length'),num_proc=5)
    collator = CSECollator(tokenizer, max_len=max_length)
    dl = DataLoader(ds_tokenized["train"],
                    batch_size=batch_size,
                    collate_fn=collator.collate,
                    drop_last=True)#重新定义取样本的函数
    return dl

# data

In [3]:
data_set_val = load_dataset('glue', 'stsb', split='validation')

Reusing dataset glue (/home/bingfa.zjq/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


# token

In [4]:
from transformers import AutoTokenizer, AutoModel
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example, max_length=32):
    return tokenizer(example["sentence1"], truncation=True,  max_length=max_length, padding='max_length')


In [5]:
data_files = './data/wiki/wiki1m_for_simcse.txt'
dataset = load_dataset('text', data_files=data_files, cache_dir="./data/")

Using custom data configuration default-a5f73cdc4a03d6e8
Reusing dataset text (./data/text/default-a5f73cdc4a03d6e8/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


In [6]:
def token_example(examples):
    total = len(examples['text'])
    sentences = examples['text'] + examples['text']
    #print(sentences)
    sent_features = tokenizer(sentences,max_length=32,truncation=True,padding="max_length")
    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]
    return features

In [7]:
from transformers.tokenization_utils_base import BatchEncoding, PaddingStrategy, PreTrainedTokenizerBase
from typing import Optional, Union, List, Dict, Tuple 
@dataclass
class OurDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    mlm: bool = True
    mlm_probability: float = 0.15

    def __call__(self, features: List[Dict[str, Union[List[int], List[List[int]], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        special_keys = ['input_ids', 'attention_mask', 'token_type_ids', 'mlm_input_ids', 'mlm_labels']
        bs = len(features)
        if bs > 0:
            num_sent = len(features[0]['input_ids'])
        else:
            return
        flat_features = []
        for feature in features:
            for i in range(num_sent):
                flat_features.append({k: feature[k][i] if k in special_keys else feature[k] for k in feature})

        batch = self.tokenizer.pad(
            flat_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )


        batch = {k: batch[k].view(bs, num_sent, -1) if k in special_keys else batch[k].view(bs, num_sent, -1)[:, 0] for k in batch}

        if "label" in batch:
            batch["labels"] = batch["label"]
            del batch["label"]
        if "label_ids" in batch:
            batch["labels"] = batch["label_ids"]
            del batch["label_ids"]

        return batch

In [8]:
train_dataset = dataset['train'].map(token_example,batched=True,remove_columns=['text'])

  0%|          | 0/1001 [00:00<?, ?ba/s]

In [9]:
train_dataset[0]['input_ids']

[[101,
  26866,
  1999,
  2148,
  2660,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  26866,
  1999,
  2148,
  2660,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [10]:
data_collator = OurDataCollatorWithPadding(tokenizer=tokenizer, mlm_probability=0.15)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True,collate_fn=data_collator)

In [11]:
len(train_loader)

15626

In [12]:
def load_dev_data(data, tokenizer): 
    sent_a = []
    sent_b = []
    score = []
    sent_a = data['sentence1']
    sent_b = data['sentence2']
    score = data['label']
   # print(len(sent_a))
    
    sent_a_encs = tokenizer(sent_a,padding=True, return_tensors="pt")
    sent_b_encs = tokenizer(sent_b, padding=True, return_tensors="pt")
    return {"sentence1": sent_a_encs, "sentence2": sent_b_encs, "label": np.array(score)}

In [13]:
dev_data = load_dev_data(data_set_val, tokenizer)

In [14]:
def compute_similarity(x, y, temp):
    cos = nn.CosineSimilarity(dim=-1)
    return cos(x, y) / temp

In [15]:
import logging

def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
            "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
        )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])
    fh = logging.FileHandler(filename, 'w')
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    
    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    
    return logger

# model

In [16]:
import imp
from model import Unsupmodel_zjq, MLPlayer
##Unsupmodel_zjq:bert+mlp+ init_weight
from transformers import AutoConfig
#from datasets import load_metric
import torch.autograd as autograd

In [17]:
import load_metric
from load_metric import Glue
metric = Glue('stsb')

In [18]:
import transformers

In [19]:
transformers.Trainer.create_optimizer_and_scheduler??

In [20]:
#transformers.optimization.get_scheduler()

In [21]:
#new metric variable stepsize
from transformers import get_linear_schedule_with_warmup
num_epochs = 1
num_training_step = num_epochs * len(dataset) # 轮数*批数，总共训练的次数

logger = get_logger('./log/stsb/对比对抗/log3.log')

In [22]:
num_epochs * len(dataset)

1

In [23]:
#print(optimizer.state_dict())

In [24]:
def eval(data, model, device):
    model.eval()
    model.to(device)
    with torch.no_grad():
        a_embed, _ = model(
            input_ids=data["sentence1"]["input_ids"].to(device),
            attention_mask=data["sentence1"]["attention_mask"].to(device),
            token_type_ids=data["sentence1"]["token_type_ids"].to(device),
        )
        b_embed, _ = model(
            input_ids=data["sentence2"]["input_ids"].to(device),
            attention_mask=data["sentence2"]["attention_mask"].to(device),
            token_type_ids=data["sentence2"]["token_type_ids"].to(device),
        )
        #cos = nn.CosineSimilarity(dim=-1)
        sim_score = F.cosine_similarity(a_embed.last_hidden_state[:,0], b_embed.last_hidden_state[:,0]).cpu().numpy()
        #print(sim_score[:20])
        #print(data['label'][:20])
        corr = scipy.stats.spearmanr(sim_score, data["label"])[0].item()
    return corr

# loss

In [25]:
loss_fct = nn.CrossEntropyLoss()

# train

In [ ]:

writer = SummaryWriter('./log/stsb/对比对抗')
PATH = './log/stsb/对比对抗/best_model1.pth'
loss_list_val = []
logger.info('start training')
min_score = 0
max_f1 = 0
#print('Debug optimi',optimizer.state_dict())


for epoch in range(num_epochs):
    #model.train()
    #for i, batch in enumerate(tqdm(dataset)):
    for i, batch in enumerate(tqdm(train_loader)):
        model.train()
        #mlp.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        #不同dropout输出
        #print(batch['input_ids'].shape)
        _, z = model(**batch)
        z_1, z_2 = z[:,0], z[:,1]
        #_, z_2 = model(**batch)
        
        #gradient = autograd.grad(loss_clean, 
              #embedding_out, retain_graph=True,allow_unused=True)[0]
        #loss_adv, logit, embedding_out, encoder_cls2 = model(flag=True, gradient=gradient, **batch)
        #经过MLP层
        #print(encoder_cls1.size())
        z_1.to(device)
        #z_1 = mlp(z_1)
        z_2.to(device)
        #print('z',z.shape)
        #print('z1',z_1.shape)
        #print('z2',z_2.shape)
        #z_2 = mlp(z_2)
        #print(z_1.unsqueeze(1).size())
        #pool1 = z_1[:64,:]
        #pool2 = z_1[64:,:]
        #print(pool1.size())        
        #计算相似度
        #loss = compute_loss(z_1, tao=0.05, device=device)

        cos_sim = compute_similarity(z_1.unsqueeze(1), z_2.unsqueeze(0), 0.05)
        label_ctr = torch.arange(cos_sim.size(0)).long().to(device)
        #print(label_ctr.size())
        #print(cos_sim.size())
        
        loss_ctr = loss_fct(cos_sim, label_ctr)
        loss = loss_ctr

        loss.backward() 
        #optimizer.clip_grad_norm(1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        optimizer.step()
        lr_scheduler.step()
        
        
        #optimizer.zero_grad()
        model.zero_grad()  
        if  i % 125 == 0:
            print('Debug!!!sche',lr_scheduler.state_dict())
            writer.add_scalar("Train/Loss", loss.item(), epoch*len(dataset)+i)
            logger.info('Epoch:[{}/{}]\t loss={:.5f}\t'.format(epoch , num_epochs, loss))
            model.eval()
            eval_score = eval(dev_data, model, device)
            writer.add_scalar("dev/score", eval_score, epoch+1)

            if eval_score > min_score:
                min_score = eval_score
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                    }, PATH)

            logger.info('Epoch:[{}]\t eval_score={}'.format(epoch , eval_score))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing Unsupmodel_zjq: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing Unsupmodel_zjq from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Unsupmodel_zjq from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Unsupmodel_zjq were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['mlp.dense.we

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-08
    lr: 3e-05
    weight_decay: 0.0

Parameter Group 1
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-08
    lr: 3e-05
    weight_decay: 0.0
)
Debug optimi {'state': {}, 'param_groups': [{'weight_decay': 0.0, 'lr': 3e-05, 'betas': (0.9, 0.999), 'eps': 1e-08, 'correct_bias': True, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]}, {'weight_decay': 0.0, 'lr': 3e-05, 'betas': (0.9, 0.999), 'eps': 1e-08, 'correct_bias': True, 'params': [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 1

  0%|          | 0/15626 [00:00<?, ?it/s][2021-08-18 16:44:31,659][1941939902.py][line:98][INFO] Epoch:[0/1]	 loss=3.71451	


Debug!!!sche {'base_lrs': [3e-05, 3e-05], 'last_epoch': 1, '_step_count': 2, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [2.999808012287214e-05, 2.999808012287214e-05], 'lr_lambdas': [None, None]}


[2021-08-18 16:44:46,401][1941939902.py][line:112][INFO] Epoch:[0]	 eval_score=0.3168717738367354
  1%|          | 125/15626 [01:40<2:56:48,  1.46it/s][2021-08-18 16:46:12,513][1941939902.py][line:98][INFO] Epoch:[0/1]	 loss=0.00256	


...

[2021-08-18 19:38:36,791][1941939902.py][line:112][INFO] Epoch:[0]	 eval_score=0.8096272775936354
 89%|████████▉ | 13875/15626 [2:55:29<19:41,  1.48it/s]  [2021-08-18 19:40:00,810][1941939902.py][line:98][INFO] Epoch:[0/1]	 loss=0.00001	


Debug!!!sche {'base_lrs': [3e-05, 3e-05], 'last_epoch': 13876, '_step_count': 13877, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [3.3597849737616796e-06, 3.3597849737616796e-06], 'lr_lambdas': [None, None]}


[2021-08-18 19:40:10,085][1941939902.py][line:112][INFO] Epoch:[0]	 eval_score=0.8088372823865296
 90%|████████▉ | 14000/15626 [2:57:03<18:22,  1.47it/s]  [2021-08-18 19:41:35,173][1941939902.py][line:98][INFO] Epoch:[0/1]	 loss=0.00009	


Debug!!!sche {'base_lrs': [3e-05, 3e-05], 'last_epoch': 14001, '_step_count': 14002, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [3.1198003327787025e-06, 3.1198003327787025e-06], 'lr_lambdas': [None, None]}


[2021-08-18 19:41:44,508][1941939902.py][line:112][INFO] Epoch:[0]	 eval_score=0.8020587846493148
 90%|█████████ | 14076/15626 [2:58:04<17:39,  1.46it/s]  

In [ ]:
tokenizer.batch_encode_plus?

In [ ]:
tokenizer?